# CS-500 Homework 2 

- Contains 3 problems, 100 pts in total
- Contains 1 extra credit problem, 30 pts in total

## Homework Instructions

1. Begin this homework by modifying the variable DREXEL_USERNAME in the next cell
    * Set the variable to your Drexel username (e.g., "db59")
2. Run all cells in this notebook before proceeding with solving problems
    * Select "Cell" -> "Run All" from the top menu bar
3. Follow instructions supplied with each problem
4. Make sure you **save your work in this notebook regularly** by pressing `CTRL+S` (`CMD+S` on macOS)
5. Submit this notebook with your saved answers

In [1]:
### CHANGE THIS TO YOUR DREXEL USERNAME!
DREXEL_USERNAME="mht47"

In [2]:
## This cell installs python (pypi) packages, then imports modules referenced in this notebook
## Make sure this cell is executed first, before you run any cell in this notebook
import ipydeps
ipydeps.pip(["wand", "json", "lxml", "redis", "sqlite3", "pandas", "numpy", "pillow", "bokeh", "sqlalchemy"])
from IPython.display import IFrame    
from wand.image import Image as WImage
from IPython.display import display
import ipywidgets as widgets
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [3]:
### we are going to use a jupyter's magick commands %sql or %%sql to run queries
##  install pip package for the sql magick:
ipydeps.pip([ "ipython-sql", "jupyter-sql", "jupyter_contrib_nbextensions"])  

## load sql magick (ipython extension)
%reload_ext sql

## %sql and %%sql magics will return pd.DataFrame objects
%config SqlMagic.autopandas=True 

## display the first 7 rows of pd.DataFrame objects
pd.set_option("max_rows", 7)

## PROBLEM 2 [30 pts]: Implementation for single ticket booking

In Problem 3 on the Midterm exam, you designed a relational schema that supports booking a ticket for passenger `Name` on a flight from `Source` to `Destination` for a given `Date`. The cheapest option must be booked when multiple flights are available. 

In this exercise (HW2, Problem 2) you will implement single ticket booking feature as a Python function below.

```python
def book_flight(Name, Date, Source, Destination):
    """
    Book a flight for a given date. The function returns True if the ticket was booked successfully, and returns False otherwise. Your relational schema (i.e., SQL tables) must be updated accordingly, but only when a ticket is booked successfully.
    
    Parameters
    ----------
    Name : string
          Passenger name
          
    Date : string
          Date of flight e.g., "2019-01-01"

    Source : string
          source city 
          
    Destination : string
          destination city 

    Returns
    -------
    success : bool
             Returns True if ticket was booked successfully, 
             returns False otherwise
             
    """
```



In [ ]:
%%sql sqlite:///routes.v2.db
    
-- Problem 2 table create statements

create table CAPACITY (
    equipment TEXT primary key,
    capacity integer not null,
    foreign key (equipment) references routes(equipment)
);

create table AIRFARE (
    ,airline TEXT not null
    ,source TEXT not null
    ,destination TEXT not null
    ,price INT not null
    ,primary key (airline, source, destination)
    ,foreign key (airline, source, destination) references routes (airline, source, destination)
);

create table SCHEDULE (
    ,airline TEXT not null
    ,source TEXT not null
    ,destination TEXT not null
    ,date DATE not null
    ,equipment TEXT not null
    ,primary key (airline, source, destination, date, equipment)
);

create table BOOKINGS (
    name TEXT not null
    ,date DATE not null
    ,source TEXT not null
    ,destination TEXT not null
    ,airline TEXT not null
    ,equipment TEXT not null
    ,price FLOAT not null
    ,primary key (name, date, source)
    ,foreign key (date, source, destination, airline, equipment) references schedule(date, source, destination, airline, equipment)
    ,foreign key (airline, source, destination, price) references AIRFARE (airline, source, destination, price)
);

In [ ]:
import pandas as pd

from sqlalchemy import create_engine
engine = create_engine('sqlite:///routes.v2.db', echo=False)

def book_flight(Name, Date, Source, Destination):
    """
    Book a flight for a given date. The function returns True if the ticket was booked successfully, and returns False otherwise. Your relational schema (i.e., SQL tables) must be updated accordingly, but only when a ticket is booked successfully.

    Parameters
    ----------
    Name : string
          Passenger name

    Date : string
          Date of flight e.g., "2019-01-01"

    Source : string
          source city 

    Destination : string
          destination city 

    Returns
    -------
    success : bool
             Returns True if ticket was booked successfully, 
             returns False otherwise

    """
    #parameters
    passenger_name = Name
    trip_date = Date
    departure = Source
    arrival = Destination
    
    
    pd.read_sql("""drop table prospective_flights; """, engine)
    
    pd.read_sql(
        """
        create table prospective_flights as
        
        select
            s.date
            ,s.source
            ,s.destination
            ,s.airline
            ,s.equipment
            ,c.capacity
            ,btd.booked
            ,a.price as price
            ,
        from SCHEDULE s 
        join AIRFARE a on s.airline = a.airline and s.source = a.source and s.destination = a.destination
        join CAPACITY c on s.equipment = c.equipment
        join
            (select b.date, b.source, b.destination, b.airline, b.equipment, count(*) as booked
            from BOOKINGS b
            where 
                b.source = :user_depart 
                and b.destination = :user_arrival 
                and b.date = :user_date
            group by b.date, b.source, b.destination, b.airline, b.equipment) btd
        on 
            s.airline = btd.airline 
            and s.equipment = btd.equipment
            and s.source = btd.source 
            and s.destination = btd.destination 
            and s.date = btd.date
        where 
            s.date = :user_date 
            and s.source = :user_depart 
            and s.destination = :user_arrival;
            
        """, engine, params={'user_date' : trip_date, 'user_depart' : departure, 'user_arrival' : arrival})
    
    flights = pd.read_sql("""
        select 
            date
            ,source
            ,destination
            ,airline
            ,equipment
            ,capacity
            ,booked
            ,price
        from prospective_flights 
        where capacity > booked
        order by price desc
        limit 1;
        """, engine)
    
    #checks whether a flight is available
    if len(flights) == 1:
        pd.read_sql(
            """
            INSERT INTO BOOKINGS(name, date, source, destination, airline, equipment, price)
            VALUES(:user_name, :user_date, :user_depart, :user_arrival, :user_airline, user_equipment, user_price);
            """, engine, params={'user_name' : passenger_name
                                ,'user_date' : flights.at[0,'date']
                                , 'user_depart' : flights.at[0,'source']
                                , 'user_arrival' : flights.at[0,'destination']
                                , 'user_airline' : flights.at[0,'airline']
                                , 'user_equipment': flights.at[0,'equipment']
                                , 'user_price': flights.at[0,'price']})
        return True
    else:
        return False
        
    
    